In [1]:
import pandas as pd
import spacy
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example

2023-06-14 12:23:45.490446: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 12:23:45.548928: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 12:23:46.355548: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Data Encoding

In [10]:
from subprocess import check_output
print(check_output(["ls", "/home/jovyan/work/data/label_data"]).decode("utf8"))

extra
ner.csv
nerReformat.csv
ner_test_data.csv
ner_train_data.csv
newner_test_data.csv
newner_train_data.csv



In [11]:
def reformat_data_spacy_format(data):
    grouped = data.groupby("sentence_idx").apply(
        lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(), s["tag"].values.tolist())])
    sentences = [s for s in grouped]
    data_encoded = []
    for s in sentences:
        full_s = ""
        start_index = 0
        last_index = 0
        index_counter = 0
        entities = []
        for w in s:
            full_s += str(w[0]) + " "
            entities.append((index_counter, index_counter + len(str(w[0])), w[1]))
            index_counter += len(str(w[0])) + 1
        data_encoded.append((full_s, {"entities": entities}))

    return data_encoded


In [16]:
train_df = pd.read_csv("/home/jovyan/work/data/label_data/newner_train_data.csv")
test_df = pd.read_csv("/home/jovyan/work/data/label_data/newner_test_data.csv")

train_data = reformat_data_spacy_format(train_df)
test_data = reformat_data_spacy_format(test_df)


# Initialize spaCy model

In [7]:
nlp = spacy.load("en_core_web_sm")

# Create the NER component
ner = nlp.get_pipe("ner")

In [8]:
# Add custom tags to the NER pipeline
for _, annotations in train_data:
    for entity in annotations.get("entities"):
        ner.add_label(entity[2])

# Disable unnecessary pipeline components
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]


In [9]:
# TRAINING THE MODEL
with nlp.disable_pipes():
    # Training iterations...
    for iteration in range(30):
        # Shuffle training examples before every iteration
        random.shuffle(train_data)
        losses = {}
        examples = []

        # Create Example objects from training data
        for text, annotation in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            examples.append(example)

        # Batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, drop=0.5, losses=losses)

        print("Losses:", losses)


/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "He says they and about 300 party supporters are tr..." with entities "[(0, 2, 'O'), (3, 7, 'O'), (8, 12, 'O'), (13, 16, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "In a telephone interview to discuss the issues at ..." with entities "[(0, 2, 'O'), (3, 4, 'O'), (5, 14, 'O'), (15, 24, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 69418.81066855586}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 52210.62648422974}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 48449.103770562484}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 45718.47410187784}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 44076.579588272805}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 42486.04913427186}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 41493.47009251388}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 40734.275653888064}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 39723.737538453875}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 39038.91492867238}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 38492.58473721899}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 37886.03152324521}
Losses: {'tok2vec': 0.0,

In [12]:
# Save the trained model
output_dir = "/home/jovyan/work/data/output/spacyModel"
nlp.to_disk(output_dir)

### Evaluation
- Evaluation Per TAG
- General Evaluation

In [13]:
#Evaluation Per TAG
test_examples = []
for text, annotation in test_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotation)
    test_examples.append(example)

# Evaluate the model on the test set
scores = nlp.evaluate(test_examples)
print("Test Scores:", scores)

# Calculate accuracy, precision, recall, and F1 score for each tag
tag_metrics = {}
for tag, values in scores['ents_per_type'].items():
    precision = round(values['p'] * 100, 2)  # Convert precision to percentage and round to 2 decimal places
    recall = round(values['r'] * 100, 2)  # Convert recall to percentage and round to 2 decimal places
    f1 = round(values['f'] * 100, 2)  # Convert F1 score to percentage and round to 2 decimal places

    tag_metrics[tag] = {'accuracy': precision, 'precision': precision, 'recall': recall, 'f1': f1}

# Print metrics for each tag
print("Metrics per Tag:")
for tag, metrics in tag_metrics.items():
    print("- Tag:", tag)
    print("  Accuracy:", metrics['accuracy'], "%")
    print("  Precision:", metrics['precision'], "%")
    print("  Recall:", metrics['recall'], "%")
    print("  F1 Score:", metrics['f1'], "%")
    print()


Test Scores: {'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'tag_acc': None, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'lemma_acc': None, 'ents_p': 0.971561122043003, 'ents_r': 0.971254217818939, 'ents_f': 0.9714076456903237, 'ents_per_type': {'O': {'p': 0.9913378139689741, 'r': 0.9931873561853718, 'f': 0.9922617232069227}, 'B-per': {'p': 0.8301941466241669, 'r': 0.8453821186190617, 'f': 0.8377192982456139}, 'I-per': {'p': 0.8675213675213675, 'r': 0.8838896952104499, 'f': 0.8756290438533428}, 'B-geo': {'p': 0.8701023209383578, 'r': 0.9098382045929019, 'f': 0.8895267253476208}, 'B-gpe': {'p': 0.9514221796100991, 'r': 0.9376377952755905, 'f': 0.9444796954314721}, 'B-org': {'p': 0.8111927642736009, 'r': 0.7334525939177102, 'f': 0.7703663937726479}, 'I-org': {'p': 0.832276474

In [19]:
entity_metrics = scores['ents_per_type']

precision_sum = 0.0
recall_sum = 0.0
f_score_sum = 0.0
num_entity_types = len(entity_metrics)

for entity_type, metrics in entity_metrics.items():
    precision_sum += metrics['p']
    recall_sum += metrics['r']
    f_score_sum += metrics['f']

general_precision = precision_sum / num_entity_types
general_recall = recall_sum / num_entity_types
general_f_score = f_score_sum / num_entity_types

print("General Precision:", general_precision)
print("General Recall:", general_recall)
print("General F-Score:", general_f_score)

General Precision: 0.8608406012098615
General Recall: 0.7423703228858265
General F-Score: 0.784462621930496
